<a href="https://colab.research.google.com/github/Sopralapanca/CommonLitChallenge/blob/TensorFlow/PrepareDataset_Chris.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [105]:
!pip install pyspellchecker
!pip install sympy

In [106]:
!git clone -b TensorFlow https://github.com/Sopralapanca/CommonLitChallenge.git

fatal: destination path 'CommonLitChallenge' already exists and is not an empty directory.


In [107]:
import pandas as pd
train_path = "/content/CommonLitChallenge/data/corrected_data.csv"
prompt_train_path = "/content/CommonLitChallenge/data/prompts_train.csv"

prompt_data = pd.read_csv(prompt_train_path, sep=',', index_col=0)
train_data = pd.read_csv(train_path, sep=',', index_col=0)
train_data = train_data.drop(columns=prompt_data.columns.tolist())
train_data.columns.tolist()
train_data = train_data[['prompt_id', 'text', 'correct_text', 'errors', 'miss', 'content', 'wording']]

In [108]:
train_data.head(2)

,prompt_id,text,correct_text,errors,miss,content,wording
0,814d6b,The third wave was an experimentto see how peo...,The third wave was an experiment see how peopl...,4,"[experimentto, eachother, experement, blindly]",0.205683,0.380538
1,814d6b,The Third Wave developed rapidly because the ...,The Third Wave developed rapidly because the ...,13,"[genuinly, acomplishments, consiquenses, activ...",3.272894,3.219757


In [109]:
prompt_data.head()

,prompt_question,prompt_title,prompt_text
prompt_id,,,
39c16e,Summarize at least 3 elements of an ideal trag...,On Tragedy,Chapter 13 \nAs the sequel to what has already...
3b9047,"In complete sentences, summarize the structure...",Egyptian Social Structure,Egyptian society was structured like a pyramid...
814d6b,Summarize how the Third Wave developed over su...,The Third Wave,Background \nThe Third Wave experiment took pl...
ebad26,Summarize the various ways the factory would u...,Excerpt from The Jungle,"With one member trimming beef in a cannery, an..."


In [110]:
import itertools
word_list = []
for prompt in prompt_data.prompt_text.tolist():
  word_list.append(prompt.replace('\n', '').split())
word_list = list(itertools.chain(*word_list))

In [111]:
from nltk.util import ngrams
from collections import Counter
import nltk
nltk.download('punkt')  # Download the required resources for tokenization

def count_ngrams(text, n):
  words = nltk.word_tokenize(text)
  ngram_counts = Counter(ngrams(words, n))
  return ngram_counts

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [112]:
del_columns = []

for n in range(2, 5):
  col = f"{n}grams-prompttext-count"
  prompt_data[col] = prompt_data.apply(lambda row: count_ngrams(row["prompt_text"], n), axis=1)

  del_columns.append(col)

  col = f"{n}grams-text-count"
  train_data[col] = train_data.apply(lambda row: count_ngrams(row["text"], n), axis=1)

  del_columns.append(col)

  col = f"{n}grams-correcttext-count"
  train_data[col] = train_data.apply(lambda row: count_ngrams(row["correct_text"], n), axis=1)

  del_columns.append(col)

prompt_data["prompt_text_length"] = prompt_data["prompt_text"].apply(len)

In [113]:
training_data = train_data.merge(prompt_data, on='prompt_id')

In [114]:
def count_cooccurring_ngrams(text, prompt_text):
    cooccurring_count = sum((text & prompt_text).values())
    return cooccurring_count

In [115]:
for n in range(2,5):
  correct_text_col = f"{n}grams-correcttext-count"
  prompt_col = f"{n}grams-prompttext-count"

  new_col=f"{n}grams-cooccurence-count"
  training_data[new_col] = training_data.apply(lambda row: count_cooccurring_ngrams(row[correct_text_col], row[prompt_col]), axis=1)

In [116]:
training_data.drop(columns=del_columns, inplace=True)

In [117]:
training_data["text_length"] = training_data["correct_text"].apply(len)
training_data["length_ratio"] = training_data["text_length"] / training_data["prompt_text_length"]

In [118]:
from spellchecker import SpellChecker

spell = SpellChecker()

spell.word_frequency.load_words(word_list)

def misspelled_counter(text):
  words = text.split()
  misspelled = spell.unknown(words)
  return len(misspelled)

def mispelled_dict(text):
  words = text.split()
  misspelled = spell.unknown(words)
  for word in misspelled:
    correction = spell.correction(words)
  return misspelled

In [119]:
training_data["misspelled_counter"] = training_data["text"].apply(lambda x: misspelled_counter(x))

In [120]:
training_data.describe()

,errors,content,wording,prompt_text_length,2grams-cooccurence-count,3grams-cooccurence-count,4grams-cooccurence-count,text_length,length_ratio,misspelled_counter
count,7165.000000,7165.000000,7165.000000,7165.000000,7165.000000,7165.000000,7165.000000,7165.000000,7165.000000,7165.000000
mean,4.765248,-0.014853,-0.063072,3875.733845,27.912352,17.371389,13.649128,411.344871,0.109941,6.224703
std,4.643054,1.043569,1.036048,784.383272,30.179689,26.146851,23.638079,302.505029,0.084691,5.397983
min,0.000000,-1.729859,-1.962614,3329.000000,0.000000,0.000000,0.000000,112.000000,0.022214,0.000000
25%,2.000000,-0.799545,-0.872720,3329.000000,9.000000,2.000000,0.000000,212.000000,0.056173,3.000000
50%,3.000000,-0.093814,-0.081769,3360.000000,18.000000,6.000000,3.000000,315.000000,0.084524,5.000000
75%,6.000000,0.499660,0.503833,5132.000000,36.000000,23.000000,19.000000,503.000000,0.132143,8.000000
max,45.000000,3.900326,4.310693,5132.000000,361.000000,335.000000,313.000000,3867.000000,1.161610,86.000000


In [121]:
import string
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import re
import nltk
from nltk.corpus import stopwords

nltk.download('stopwords')
nltk.download('wordnet')
stop_words = stopwords.words('english')
lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer()

def preprocessText(text):
    # replace newline with space
    text = text.replace("\n", "")

    text = text.replace('\r', '')
    # Replace curly apostrophe with straight single quote
    text = text.replace('’', "'")

    # Normalize spaces around punctuation marks
    #text = re.sub(r'\s+', ' ', text)
    #text = re.sub(r'\s([.,!?])', r'\1', text)
    #text = re.sub(r'([.,!?])\s', r'\1', text)
    text = text.strip()

    # lower case
    text = text.lower()

    # split text
    words = text.split()

    # stop word removal
    words = [w for w in words if not w in stop_words]

    # stemming
    #words = [stemmer.stem(w) for w in words]

    # lemmatization
    words = [lemmatizer.lemmatize(w) for w in words]

    text = ' '.join(words)

    return text

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [122]:
def add_row(df1, df2):
  row = df2.unique().tolist()[0]
  row = preprocessText(row)
  combined_data = pd.concat([pd.Series([row]),df1.loc[:]]).reset_index(drop=True) #append row on the head of the dataframe
  return combined_data

In [123]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Group by 'prompt_id' and compute TF-IDF separately for each class
tfidf_vectorizers = {}

for class_id, group in training_data.groupby('prompt_id'):
    text_data = group['correct_text'].apply(preprocessText)


    prompt_question_data = group['prompt_question']
    prompt_title_data = group['prompt_title']
    prompt_text_data = group['prompt_text']

    # Concatenate the preprocessed data for TF-IDF calculation
    combined_data = add_row(text_data, prompt_question_data)
    combined_data = add_row(combined_data, prompt_title_data)
    combined_data = add_row(combined_data, prompt_text_data)


    # Compute TF-IDF
    tfidf_vectorizer = TfidfVectorizer()
    tfidf_matrix = tfidf_vectorizer.fit_transform(combined_data)
    tfidf_vectorizers[class_id] = {'vectorizer': tfidf_vectorizer, 'matrix': tfidf_matrix}

In [124]:
import sympy

karp_tfidf_score = {}

# Calculate TF-IDF scores for each document
for class_id, group in training_data.groupby('prompt_id'):
    tfidf_vectorizer = tfidf_vectorizers[class_id]['vectorizer']
    tfidf_matrix = tfidf_vectorizers[class_id]['matrix']
    tfidf_matrix = tfidf_matrix[3:] #remove first 3 rows f the matrix since they belongs to prompt_text, prompt_question, prompt_title
    modulus = sympy.randprime(tfidf_matrix.shape[0], tfidf_matrix.shape[1])
    # Iterate through documents and calculate TF-IDF scores
    for index, row in group.iterrows():
        doc_tfidf = tfidf_matrix[index - group.index[0]].toarray()[0]

        doc_tfidf = doc_tfidf[doc_tfidf>0]

        # Calculate the average TF-IDF score for the document
        gamma = 1e-2
        single_tfidf_score = sum([t**(gamma*i) for i, t in enumerate(doc_tfidf)]) % modulus

        karp_tfidf_score[index] = single_tfidf_score

In [125]:
# Add the calculated average TF-IDF scores as a new column to the DataFrame
training_data['karp_tfidf_score'] = [karp_tfidf_score[index] for index in training_data.index]

In [126]:
# normalize the data taking into consideration the prompt title

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

def normalize_col(training_data, col):
  # Create a new DataFrame to store the normalized text length values
  normalized_df = pd.DataFrame()

  # Group by 'prompt_title' and apply the normalization separately for each group
  for title, group in training_data.groupby('prompt_title'):
      normalized_text_length = scaler.fit_transform(group[[col]])
      new_name = "normalized_"+col
      group[new_name] = normalized_text_length
      normalized_df = pd.concat([normalized_df, group])
  training_data = normalized_df.copy()
  return training_data

In [127]:
normalize_cols = ["text_length", "misspelled_counter", "2grams-cooccurence-count", "3grams-cooccurence-count", "4grams-cooccurence-count", "karp_tfidf_score"]
for col in normalize_cols:
  training_data = normalize_col(training_data, col)

In [128]:
training_data.head(4)

,prompt_id,text,correct_text,errors,miss,content,wording,prompt_question,prompt_title,prompt_text,...,text_length,length_ratio,misspelled_counter,karp_tfidf_score,normalized_text_length,normalized_misspelled_counter,normalized_2grams-cooccurence-count,normalized_3grams-cooccurence-count,normalized_4grams-cooccurence-count,normalized_karp_tfidf_score
3099,3b9047,"In Egypt, there were many occupations and soci...","In Egypt, there were many occupations and soci...",13,"[Ra, pharaohs, pharaohs, vizier, vizier, quell...",3.128928,4.231226,"In complete sentences, summarize the structure...",Egyptian Social Structure,Egyptian society was structured like a pyramid...,...,1345,0.404025,23,38.000761,0.328362,0.267442,0.141618,0.056426,0.030717,0.962679
3100,3b9047,The highest class was Pharaohs these people we...,The highest class was that these people were g...,3,"[Pharaohs, gonvener, vizier]",-0.210614,-0.471415,"In complete sentences, summarize the structure...",Egyptian Social Structure,Egyptian society was structured like a pyramid...,...,151,0.045359,4,11.939888,0.010386,0.046512,0.011561,0.006270,0.003413,0.184543
3101,3b9047,The Egyptian society is really different from ...,The vampire society is really different from o...,3,"[Egyptian, socicties, wyas]",0.205683,0.380538,"In complete sentences, summarize the structure...",Egyptian Social Structure,Egyptian society was structured like a pyramid...,...,452,0.135777,5,22.704046,0.090546,0.058140,0.020231,0.000000,0.000000,0.505944
3102,3b9047,We have the gods and then Logan and If Logan ...,We have the gods and then Logan and If Logan ...,3,"[Logan, wh, arund]",-1.547163,-1.461245,"In complete sentences, summarize the structure...",Egyptian Social Structure,Egyptian society was structured like a pyramid...,...,129,0.038750,3,7.538979,0.004527,0.034884,0.002890,0.000000,0.000000,0.053139


In [129]:
training_data.corrwith(training_data["content"])

<ipython-input-129-3ecd97e2210c>:1: FutureWarning: The default value of numeric_only in DataFrame.corrwith is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  training_data.corrwith(training_data["content"])


errors                                 0.523034
content                                1.000000
wording                                0.751380
prompt_text_length                    -0.038532
2grams-cooccurence-count               0.515232
3grams-cooccurence-count               0.373884
4grams-cooccurence-count               0.324842
text_length                            0.798812
length_ratio                           0.778609
misspelled_counter                     0.689731
karp_tfidf_score                       0.834647
normalized_text_length                 0.784119
normalized_misspelled_counter          0.691504
normalized_2grams-cooccurence-count    0.546632
normalized_3grams-cooccurence-count    0.390053
normalized_4grams-cooccurence-count    0.335660
normalized_karp_tfidf_score            0.838755
dtype: float64

In [130]:
training_data.corrwith(training_data["wording"])

<ipython-input-130-360ae61d465e>:1: FutureWarning: The default value of numeric_only in DataFrame.corrwith is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  training_data.corrwith(training_data["wording"])


errors                                 0.249451
content                                0.751380
wording                                1.000000
prompt_text_length                    -0.121241
2grams-cooccurence-count               0.152592
3grams-cooccurence-count               0.023726
4grams-cooccurence-count              -0.008348
text_length                            0.543625
length_ratio                           0.549764
misspelled_counter                     0.516190
karp_tfidf_score                       0.567776
normalized_text_length                 0.559794
normalized_misspelled_counter          0.559061
normalized_2grams-cooccurence-count    0.219734
normalized_3grams-cooccurence-count    0.059694
normalized_4grams-cooccurence-count    0.018503
normalized_karp_tfidf_score            0.574807
dtype: float64

In [131]:
training_data.to_csv('dataset.csv', index=False)